In [1]:
import numpy as np
import pandas as pd

In [2]:
# read some dataframes
df = pd.read_csv('data/player_valuations_with_age_and_club.csv')
managers = pd.read_csv('data/manager_data.csv')
managers = managers[['club_id', 'manager', 'first_game', 'last_game']]

In [3]:
# add a manager column to the players df (this takes a while)
df['manager'] = None

for index, row in managers.iterrows():
    group_by_id = row['club_id']
    manager = row['manager']
    fg = row['first_game']
    lg = row['last_game']

    df.loc[(df['date'] >= fg) & (df['date'] <= lg) &
           (df['player_club_id'] == group_by_id), 'manager'] = manager

In [4]:
# add league id column to the players df
clubs = pd.read_csv('data/clubs.csv')
clubs = clubs[['club_id', 'domestic_competition_id', 'name']]
clubs = clubs.rename(columns={'club_id': 'player_club_id', 'name': 'club_name'})
df = df.merge(clubs, on='player_club_id', how='left')

In [5]:
# the first valuations don't have a club id and manager yet, but i don't think this is a problem
df

,player_id,name,age_at_valuation,position,height_in_cm,foot,market_value_in_eur,date,player_club_id,manager,domestic_competition_id,club_name
0,11757,Adam Kwarasey,24.553046,Goalkeeper,190.0,right,750000,2012-07-01,NaN,None,NaN,NaN
1,22474,Martin Örnskov,26.724162,Midfield,185.0,left,1000000,2012-07-01,NaN,None,NaN,NaN
2,24112,Rune Jarstein,27.753593,Goalkeeper,192.0,right,1000000,2012-07-01,NaN,None,NaN,NaN
3,26751,Johan Björdal,26.157426,Defender,188.0,right,750000,2012-07-01,NaN,None,NaN,NaN
4,37287,Tarik Elyounoussi,24.353183,Attack,172.0,right,1500000,2012-07-01,NaN,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
351247,896154,Leonid Mina,19.482546,Attack,182.0,right,100000,2023-09-21,2671.0,None,GR1,PAS Giannina
351248,932259,Konstantinos Chrysopoulos,20.336756,Defender,193.0,left,500000,2023-09-21,2441.0,None,GR1,AEK Athens
351249,950091,Stefanos Tzimas,17.705681,Attack,184.0,both,1000000,2023-09-21,1091.0,Razvan Lucescu,GR1,PAOK Thessaloniki
351250,1166093,Georgios Konstantakopoulos,18.724162,Midfield,182.0,both,50000,2023-09-21,30120.0,Giannis Anastasiou,GR1,AE Kifisias


In [6]:
def get_value_increase(df, group_by_id, group_by='player_club_id', max_age_at_start=21,
                       max_age=24, min_stay=1, position="All", debug=False):
    """
    Get the median percentage increase in value per year for a group of players that meet the criteria:
    - their manager/club_id/domestic_competition_id matches the group_by_id
    - max_age_at_start: the maximum age at which the player started at the club/league/manager
    - max_age: keep only the player valuations that are below this age
    - min_stay: the minimum number of years the player has to stay at the club/league/manager
    - position: the position of the player

    - debug is True if you want to return the dataframe of selected players instead of the median increase
    """
    assert group_by in ['player_club_id', 'manager', 'domestic_competition_id']

    # select only the player valuations that meet the criteria
    all_valuations = df[df[group_by] == group_by_id]
    all_valuations = all_valuations[all_valuations['age_at_valuation'] < max_age]
    # sort by date
    all_valuations = all_valuations.sort_values(by='date')

    # keep only the players that match the given position
    if position != 'All':
        all_valuations = all_valuations[all_valuations['position'] == position]

    # group by player and get the first and last market value and age
    players = all_valuations.groupby(['player_id', 'name', 'position']).agg(
        {'market_value_in_eur': ['first', 'last'], 'age_at_valuation': ['first', 'last']})
    players.columns = ['first_market_value_in_eur', 'last_market_value_in_eur',
                       'first_age_at_valuation', 'last_age_at_valuation']
    players = players.reset_index()

    # compute the age difference
    players['age_diff'] = players['last_age_at_valuation'] - \
                          players['first_age_at_valuation']

    # remove players that have stayed less than min_stay years
    players = players[players['age_diff'] > min_stay]
    # remove players that started at an age higher than max_age_at_start
    players = players[players['first_age_at_valuation'] < max_age_at_start]

    # remove players that have less than 10 valuations (or 6 if position is not All)
    if position == 'All':
        if len(players) < 10:
            return None
    else:
        if len(players) < 6:
            return None

    # compute the percentage increase in value
    players['market_value_diff_percent'] = players['last_market_value_in_eur'] / \
                                           players['first_market_value_in_eur']

    # do the age_diff-th root of the percentage increase to get a comparison that is comparable per year
    players['market_value_diff_percent_per_year'] = players['market_value_diff_percent'].abs() ** \
                                                    (1 / players['age_diff'])
    # change NaN to -1 (idk if it is still needed)
    players['market_value_diff_percent_per_year'] = players['market_value_diff_percent_per_year'].fillna(-1)

    if debug:
        return players

    # return the median increase per year (mean is not good, because it is skewed by outliers like Mbappe)
    median_increase_per_year = players['market_value_diff_percent_per_year'].median()

    return median_increase_per_year


In [7]:
def get_value_increase_df(group_by):
    value_increases = []
    # skip Goalkeeper because there are too few players
    positions = ['All', 'Defender', 'Midfield', 'Attack']


    for position in positions:
        for group_by_id in df[group_by].unique():
            if group_by_id != group_by_id: # this apparently skips NaNs
                continue

            value_increase = get_value_increase(df, group_by_id, group_by=group_by, max_age_at_start=21,
                                                max_age=24, min_stay=1, position=position, debug=False)
            if value_increase is None:
                continue

            if group_by == 'player_club_id':
                club_name = clubs[clubs['player_club_id'] == group_by_id]['club_name'].values[0]
                row = (position, group_by_id, club_name, value_increase)
            else:
                row = (position, group_by_id, value_increase)

            print(row)
            value_increases.append(row)

    if group_by == 'player_club_id':
        value_increases = pd.DataFrame(value_increases, columns=['position', group_by, 'club_name', 'value_increase'])
    else:
        value_increases = pd.DataFrame(value_increases, columns=['position', group_by, 'value_increase'])
    return value_increases

club_value_increases = get_value_increase_df('player_club_id')
manager_value_increases = get_value_increase_df('manager')
league_value_increases = get_value_increase_df('domestic_competition_id')

('All', 317.0, 'Twente Enschede FC', 1.1673817899094063)
('All', 190.0, 'FC Copenhagen', 1.6367144297484437)
('All', 865.0, 'FC Midtjylland', 1.6211678516411085)
('All', 2778.0, 'FC Nordsjaelland', 1.5981919504942883)
('All', 2414.0, 'AC Horsens', 1.409835869472015)
('All', 173.0, 'Odense Boldklub', 1.3247118617355953)
('All', 1177.0, 'Silkeborg IF', 1.6069478132847357)
('All', 5724.0, 'Randers FC', 1.3887010051876334)
('All', 678.0, 'Aarhus GF', 1.4387355457844178)
('All', 5817.0, 'SönderjyskE', 1.3681927838700618)
('All', 1053.0, 'Aalborg BK', 1.537684124497407)
('All', 206.0, 'Bröndby IF', 1.452552493370532)
('All', 3426.0, 'Esbjerg fB', 1.3744717366370196)
('All', 2578.0, 'St. Johnstone FC', 1.704262523948659)
('All', 964.0, 'Zenit St. Petersburg', 1.1681847800914817)
('All', 1083.0, 'FK Rostov', 1.1826045919888022)
('All', 3725.0, 'Akhmat Grozny', 1.2012780511110464)
('All', 121.0, 'Dynamo Moscow', 1.374558116585228)
('All', 932.0, 'Lokomotiv Moscow', 1.5937048988795457)
('All', 3

In [17]:
# keep best 10 for each position in the df
def keep_n_best(df, n=10, worst=False):
    best = []
    for position in ['All', 'Defender', 'Midfield', 'Attack']:
        best.append(df[df['position'] == position].
                    sort_values(by='value_increase', ascending=worst).head(n))
    best = pd.concat(best)
    return best

best_clubs = keep_n_best(club_value_increases)
worst_leagues = keep_n_best(club_value_increases, worst=True)

In [18]:
club_value_increases

,position,player_club_id,club_name,value_increase
0,All,317.0,Twente Enschede FC,1.167382
1,All,190.0,FC Copenhagen,1.636714
2,All,865.0,FC Midtjylland,1.621168
3,All,2778.0,FC Nordsjaelland,1.598192
4,All,2414.0,AC Horsens,1.409836
...,...,...,...,...
547,Attack,2503.0,Boavista FC,1.382625
548,Attack,29228.0,Royal Excel Mouscron (-2022),1.147089
549,Attack,347.0,FC Metz,1.142615
550,Attack,1420.0,Angers SCO,1.908990


In [19]:
# check the Nantes midfield players to see if they are really that good
get_value_increase(df, 995, group_by='player_club_id', max_age_at_start=21, max_age=24, min_stay=1,
                   position='Midfield', debug=True)

,player_id,name,position,first_market_value_in_eur,last_market_value_in_eur,first_age_at_valuation,last_age_at_valuation,age_diff,market_value_diff_percent,market_value_diff_percent_per_year
1,164771,Jordan Veretout,Midfield,4500000,8000000,20.873374,22.264203,1.390828,1.777778,1.512380
3,181382,Abdoulaye Touré,Midfield,150000,5000000,20.429843,23.895962,3.466119,33.333333,2.750181
8,228999,Alexis Alégué,Midfield,150000,400000,18.885695,23.885010,4.999316,2.666667,1.216761
15,332889,Valentin Rongier,Midfield,250000,12000000,20.104038,23.748118,3.644079,48.000000,2.893093
19,463611,Abdoulaye Dabo,Midfield,1000000,500000,17.508556,21.314168,3.805613,0.500000,0.833486
20,476527,Kenny Rocha Santos,Midfield,150000,1500000,19.414100,21.475702,2.061602,10.000000,3.055342
21,635336,Imrân Louza,Midfield,300000,12000000,20.232717,22.094456,1.861739,40.000000,7.253010


In [20]:
# save the dfs

best_clubs["position"]= best_clubs["position"].replace("Midfield","Midfielder")
best_clubs["position"]= best_clubs["position"].replace("Attack","Attacker")
best_clubs.to_csv("data/club_value_increase.csv")

In [19]:
worst_leagues.to_csv("data/club_value_increase.csv")